In [76]:
import pandas as pd
import sys  
sys.path.insert(0, '../scripts')
import clean_data
import utilities
import loading_data
import norm_scaling
import seaborn as sn
import numpy as np

In [2]:
#loaded file
df= loading_data.load_csv('../data/data.csv')

/home/dibora/User-Analytics-in-the-Telecommunication-Industry/venv_usr_analytics/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  


In [3]:
df.head()

,Bearer Id,Start,Start ms,End,End ms,Dur. (ms),IMSI,MSISDN/Number,IMEI,Last Location Name,...,Youtube DL (Bytes),Youtube UL (Bytes),Netflix DL (Bytes),Netflix UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes)
0,"13,114,483,460,844,900,352.00",4/4/2019 12:01,770.00,4/25/2019 14:35,662.00,"1,823,652.00","208,201,448,079,117.00","33,664,962,239.00","35,521,209,507,511.00",9.16456699548519E+015,...,"15,854,611.00","2,501,332.00","8,198,936.00","9,656,251.00","278,082,303.00","14,344,150.00","171,744,450.00","8,814,393.00","36,749,741.00","308,879,636.00"
1,"13,114,483,482,878,900,224.00",4/9/2019 13:04,235.00,4/25/2019 8:15,606.00,"1,365,104.00","208,201,909,211,140.00","33,681,854,413.00","35,794,009,006,359.00",L77566A,...,"20,247,395.00","19,111,729.00","18,338,413.00","17,227,132.00","608,750,074.00","1,170,709.00","526,904,238.00","15,055,145.00","53,800,391.00","653,384,965.00"
2,"13,114,483,484,080,500,736.00",4/9/2019 17:42,1.00,4/25/2019 11:58,652.00,"1,361,762.00","208,200,314,458,056.00","33,760,627,129.00","35,281,510,359,387.00",D42335A,...,"19,725,661.00","14,699,576.00","17,587,794.00","6,163,408.00","229,584,621.00","395,630.00","410,692,588.00","4,215,763.00","27,883,638.00","279,807,335.00"
3,"13,114,483,485,442,799,616.00",4/10/2019 0:31,486.00,4/25/2019 7:36,171.00,"1,321,509.00","208,201,402,342,131.00","33,750,343,200.00","35,356,610,164,913.00",T21824A,...,"21,388,122.00","15,146,643.00","13,994,646.00","1,097,942.00","799,538,153.00","10,849,722.00","749,039,933.00","12,797,283.00","43,324,218.00","846,028,530.00"
4,"13,114,483,499,480,700,928.00",4/12/2019 20:10,565.00,4/25/2019 10:40,954.00,"1,089,009.00","208,201,401,415,120.00","33,699,795,932.00","35,407,009,745,539.00",D88865A,...,"15,259,380.00","18,962,873.00","17,124,581.00","415,218.00","527,707,248.00","3,529,801.00","550,709,500.00","13,910,322.00","38,542,814.00","569,138,589.00"


In [4]:
#handle missing values, TCP retransmitted column contains more than 30% of null values but we won't drop it since we need it for analyzing the experience of users. instead we fix the missing values.
cleaned_data = clean_data.Handle_missing_values(df,drop_rows=False,drop_cols=False)
cleaned_data.isnull().sum()

['Start', 'End', 'Last Location Name', 'Handset Manufacturer', 'Handset Type']


Bearer Id                                   0
Start                                       0
Start ms                                    0
End                                         0
End ms                                      0
Dur. (ms)                                   0
IMSI                                        0
MSISDN/Number                               0
IMEI                                        0
Last Location Name                          0
Avg RTT DL (ms)                             0
Avg RTT UL (ms)                             0
Avg Bearer TP DL (kbps)                     0
Avg Bearer TP UL (kbps)                     0
TCP DL Retrans. Vol (Bytes)                 0
TCP UL Retrans. Vol (Bytes)                 0
DL TP < 50 Kbps (%)                         0
50 Kbps < DL TP < 250 Kbps (%)              0
250 Kbps < DL TP < 1 Mbps (%)               0
DL TP > 1 Mbps (%)                          0
UL TP < 10 Kbps (%)                         0
10 Kbps < UL TP < 50 Kbps (%)     

Aggregate, per customer, the following information (treat missing & outliers by replacing by the mean or the mode of the corresponding variable):
Average TCP retransmission
Average RTT
Handset type
Average throughput


In [5]:
# aggregate of Average throughput UL for each user
utilities.find_agg(cleaned_data, "MSISDN/Number","Avg Bearer TP UL (kbps)" ,"sum", "Avg throughput UL per user", 10, order=False )

,MSISDN/Number,Avg throughput UL per user
53644,"33,663,706,799.00","1,832,273.00"
29539,"33,658,124,130.00","167,024.00"
29313,"33,658,075,438.00","163,585.00"
100319,"33,763,744,937.00","161,563.00"
69910,"33,668,708,263.00","159,430.00"
25134,"33,650,520,779.00","145,320.00"
99990,"33,763,586,476.00","138,359.00"
39898,"33,660,394,384.00","137,330.00"
36257,"33,659,546,392.00","127,412.00"
66417,"33,667,600,457.00","127,200.00"


In [6]:
# aggregate of Average throughput DL for each user
utilities.find_agg(cleaned_data, "MSISDN/Number","Avg Bearer TP DL (kbps)" ,"sum", "Avg throughput DL per user", 10, order=False )

,MSISDN/Number,Avg throughput DL per user
53644,"33,663,706,799.00","16,231,020.00"
97584,"33,762,333,464.00","846,163.00"
69910,"33,668,708,263.00","680,994.00"
36257,"33,659,546,392.00","670,930.00"
69020,"33,668,425,947.00","658,687.00"
84486,"33,698,174,760.00","606,931.00"
73146,"33,669,946,573.00","581,209.00"
13141,"33,625,700,673.00","556,109.00"
31726,"33,658,581,169.00","510,852.00"
74585,"33,672,505,906.00","480,466.00"


In [7]:
# aggregate of Average RTT UL for each user
utilities.find_agg(cleaned_data, "MSISDN/Number","Avg RTT UL (ms)" ,"sum", "Avg RTT UL per user", 10, order=False )

,MSISDN/Number,Avg RTT UL per user
53644,"33,663,706,799.00","19,577.00"
37287,"33,659,778,586.00","7,120.00"
17134,"33,633,298,261.00","5,607.00"
94139,"33,761,092,970.00","5,422.00"
65118,"33,667,163,239.00","5,414.00"
14494,"33,627,850,141.00","5,295.00"
96962,"33,762,057,942.00","5,248.00"
69218,"33,668,497,162.00","4,682.00"
18351,"33,636,368,478.00","4,026.00"
103348,"33,778,078,390.00","3,923.00"


In [8]:
# aggregate of Average RTT DL for each user
utilities.find_agg(cleaned_data, "MSISDN/Number","Avg RTT DL (ms)" ,"sum", "Avg RTT DL per user", 10, order=False )

,MSISDN/Number,Avg RTT DL per user
53644,"33,663,706,799.00","295,644.00"
47983,"33,662,317,023.00","96,923.00"
41944,"33,660,874,265.00","64,666.00"
80249,"33,683,692,867.00","54,847.00"
85683,"33,698,551,167.00","45,923.00"
96414,"33,761,813,523.00","36,977.00"
70142,"33,668,791,629.00","36,125.00"
93844,"33,760,941,100.00","26,971.00"
1713,"33,606,788,933.00","26,250.00"
74240,"33,671,816,754.00","25,602.00"


In [9]:
# aggregate of Handset types of each user
utilities.find_agg(cleaned_data, "MSISDN/Number","Handset Type" ,"count", "Handset type per user", 10, order=False )



,MSISDN/Number,Handset type per user
53644,"33,663,706,799.00",1067
13526,"33,626,320,676.00",18
6437,"33,614,892,860.00",17
13180,"33,625,779,332.00",17
37052,"33,659,725,664.00",16
76363,"33,675,877,202.00",15
92923,"33,760,536,639.00",15
65118,"33,667,163,239.00",13
1279,"33,604,515,716.00",12
92577,"33,760,413,819.00",12


In [10]:
# aggregate of TCP retransmission DL of each user
utilities.find_agg(cleaned_data, "MSISDN/Number","TCP DL Retrans. Vol (Bytes)" ,"sum", "Avg TCP retransmission per user", 10, order=False )

,MSISDN/Number,Avg TCP retransmission per user
53644,"33,663,706,799.00","10,558,221,768.00"
66860,"33,667,725,464.00","8,332,732,777.00"
34123,"33,659,084,281.00","7,938,178,623.00"
42366,"33,660,977,522.00","7,374,100,838.00"
94681,"33,761,284,050.00","6,039,191,773.00"
98213,"33,762,644,658.00","5,878,002,490.00"
25774,"33,650,658,056.00","5,590,343,967.00"
51638,"33,663,180,885.00","5,562,845,158.00"
99929,"33,763,572,381.00","5,178,607,391.00"
31965,"33,658,638,625.00","5,165,784,763.00"


In [11]:
# aggregate of TCP retransmission UL of each user
utilities.find_agg(cleaned_data, "MSISDN/Number","TCP UL Retrans. Vol (Bytes)" ,"sum", "Avg TCP retransmission per user", 10, order=False )

,MSISDN/Number,Avg TCP retransmission per user
70913,"33,669,054,076.00","5,168,607,636.50"
30175,"33,658,249,284.00","2,637,302,536.00"
100005,"33,763,588,772.00","2,455,599,840.00"
57181,"33,664,698,321.00","1,751,383,517.00"
58664,"33,665,090,461.00","1,727,810,000.00"
93811,"33,760,918,898.00","1,237,885,767.50"
68743,"33,668,333,080.00","807,925,207.00"
45158,"33,661,679,531.00","794,825,766.00"
85123,"33,698,382,849.00","719,571,807.00"
69068,"33,668,446,094.00","526,246,337.00"


Compute & list 10 of the top, bottom and most frequent:
TCP values in the dataset. 
RTT values in the dataset.
Throughput values in the dataset.


### Top bottom and most frequent Throuput values in the dataset

In [46]:
#top 10 throughput UL values in the dataset
top_throughput_UL = pd.DataFrame({"Top TP UL": cleaned_data["Avg Bearer TP UL (kbps)"].unique()}).sort_values(by="Top TP UL",ascending=False).reset_index().head(10)
#bottom 10 throughput UL values in the dataset
bottom_throughput_UL = pd.DataFrame({"Bottom TP UL": cleaned_data["Avg Bearer TP UL (kbps)"].unique()}).sort_values(by="Bottom TP UL",ascending=True).reset_index().head(10)
#top 10 frequent throughput UL values in the dataset
frequent_throughput_UL = utilities.find_agg(cleaned_data, "Avg Bearer TP UL (kbps)","Avg Bearer TP UL (kbps)" ,"count", "frequency of TP ULs", 10, order=False )

print("Top 10 throughput UL in the dataset")
print(top_throughput_UL,"\n")
print("Bottom 10 throughput UL in the dataset")
print(bottom_throughput_UL,"\n")
print("Top 10 frequent throughput UL in the dataset")
print(frequent_throughput_UL)
print("\n")

#top 10 throughput DL values in the dataset
top_throughput_DL = pd.DataFrame({"Top TP DL": cleaned_data["Avg Bearer TP DL (kbps)"].unique()}).sort_values(by="Top TP DL",ascending=False).reset_index().head(10)
#bottom 10 throughput UL values in the dataset
bottom_throughput_DL = pd.DataFrame({"Bottom TP DL": cleaned_data["Avg Bearer TP DL (kbps)"].unique()}).sort_values(by="Bottom TP DL",ascending=True).reset_index().head(10)
#top 10 frequent throughput UL values in the dataset
frequent_throughput_DL = utilities.find_agg(cleaned_data, "Avg Bearer TP DL (kbps)","Avg Bearer TP DL (kbps)" ,"count", "frequency of TP DLs", 10, order=False )

print("Top 10 throughput DL in the dataset")
print(top_throughput_DL,"\n")
print("Bottom 10 throughput DL in the dataset")
print(bottom_throughput_DL,"\n")
print("Top 10 frequent throughput DL in the dataset")
print(frequent_throughput_DL)

Top 10 throughput UL in the dataset
   index  Top TP UL
0  11300  58,613.00
1   3204  57,792.00
2  13512  57,750.00
3  12264  57,006.00
4  11798  56,763.00
5  11203  54,032.00
6   7987  53,572.00
7   4043  52,640.00
8  11229  52,084.00
9  13153  51,562.00 

Bottom 10 throughput UL in the dataset
   index  Bottom TP UL
0     78          0.00
1   1597          1.00
2      9          2.00
3    368          3.00
4     24          4.00
5    207          5.00
6    252          6.00
7     53          7.00
8     98          8.00
9      2          9.00 

Top 10 frequent throughput UL in the dataset
    Avg Bearer TP UL (kbps)  frequency of TP ULs
40                    40.00                 7794
54                    54.00                 6344
47                    47.00                 5840
53                    53.00                 4835
9                      9.00                 3812
52                    52.00                 3786
51                    51.00                 3646
44         

### Top bottom and most frequent RTT values in the dataset

In [48]:
#top 10 RTT UL values in the dataset
top_RTT_UL = pd.DataFrame({"Top RTT UL": cleaned_data["Avg RTT UL (ms)"].unique()}).sort_values(by="Top RTT UL",ascending=False).reset_index().head(10)
#bottom 10 throughput UL values in the dataset
bottom_RTT_UL = pd.DataFrame({"Bottom RTT UL": cleaned_data["Avg RTT UL (ms)"].unique()}).sort_values(by="Bottom RTT UL",ascending=True).reset_index().head(10)
#top 10 frequent throughput UL values in the dataset
frequent_RTT_UL = utilities.find_agg(cleaned_data, "Avg RTT UL (ms)","Avg RTT UL (ms)" ,"count", "frequency of TP UL", 10, order=False )

print("Top 10 RTT UL in the dataset")
print(top_RTT_UL,"\n")
print("Bottom 10 RTT UL in the dataset")
print(bottom_RTT_UL,"\n")
print("Top 10 frequent RTT UL in the dataset")
print(frequent_RTT_UL)
print("\n")

#top 10 throughput DL values in the dataset
top_RTT_DL = pd.DataFrame({"Top RTT DL": cleaned_data["Avg RTT DL (ms)"].unique()}).sort_values(by="Top RTT DL",ascending=False).reset_index().head(10)
#bottom 10 throughput UL values in the dataset
bottom_RTT_DL = pd.DataFrame({"Bottom RTT DL": cleaned_data["Avg RTT DL (ms)"].unique()}).sort_values(by="Bottom RTT DL",ascending=True).reset_index().head(10)
#top 10 frequent throughput UL values in the dataset
frequent_RTT_DL = utilities.find_agg(cleaned_data, "Avg RTT DL (ms)","Avg RTT DL (ms)" ,"count", "frequency of RTT DL", 10, order=False )

print("Top 10 RTT DL in the dataset")
print(top_RTT_DL,"\n")
print("Bottom 10 RTT DL in the dataset")
print(bottom_RTT_DL,"\n")
print("Top 10 frequent RTT DL in the dataset")
print(frequent_RTT_DL)

Top 10 RTT UL in the dataset
   index  Top RTT UL
0    303    7,120.00
1    663    5,607.00
2    543    5,422.00
3     41    5,279.00
4    567    4,677.00
5    404    4,226.00
6    437    4,021.00
7    674    3,923.00
8    627    3,093.00
9    423    3,029.00 

Bottom 10 RTT UL in the dataset
   index  Bottom RTT UL
0     14           0.00
1      8           1.00
2      5           2.00
3     18           3.00
4      3           4.00
5      0           5.00
6      7           6.00
7      2           7.00
8     40           8.00
9     44           9.00 

Top 10 frequent RTT UL in the dataset
   Avg RTT UL (ms)  frequency of TP UL
5             5.00               42475
2             2.00               14277
1             1.00               12826
4             4.00               10242
3             3.00                7935
6             6.00                7518
7             7.00                4196
0             0.00                4126
8             8.00                3209
9           

### Top bottom and most frequent TCP retransmission values in the dataset

In [49]:
#top 10 RTT UL values in the dataset
top_TCP_UL = pd.DataFrame({"Top TCP UL": cleaned_data["TCP UL Retrans. Vol (Bytes)"].unique()}).sort_values(by="Top TCP UL",ascending=False).reset_index().head(10)
#bottom 10 throughput UL values in the dataset
bottom_TCP_UL = pd.DataFrame({"Bottom TCP UL": cleaned_data["TCP UL Retrans. Vol (Bytes)"].unique()}).sort_values(by="Bottom TCP UL",ascending=True).reset_index().head(10)
#top 10 frequent throughput UL values in the dataset
frequent_TCP_UL = utilities.find_agg(cleaned_data, "TCP UL Retrans. Vol (Bytes)","TCP UL Retrans. Vol (Bytes)" ,"count", "frequency of TCP UL", 10, order=False )

print("Top 10 TCP UL in the dataset")
print(top_RTT_UL,"\n")
print("Bottom 10 TCP UL in the dataset")
print(bottom_RTT_UL,"\n")
print("Top 10 frequent TCP UL in the dataset")
print(frequent_RTT_UL)
print("\n")

#top 10 throughput DL values in the dataset
top_TCP_DL = pd.DataFrame({"Top TCP DL": cleaned_data["TCP DL Retrans. Vol (Bytes)"].unique()}).sort_values(by="Top TCP DL",ascending=False).reset_index().head(10)
#bottom 10 throughput UL values in the dataset
bottom_TCP_DL = pd.DataFrame({"Bottom TCP DL": cleaned_data["TCP DL Retrans. Vol (Bytes)"].unique()}).sort_values(by="Bottom TCP DL",ascending=True).reset_index().head(10)
#top 10 frequent throughput UL values in the dataset
frequent_TCP_DL = utilities.find_agg(cleaned_data, "TCP DL Retrans. Vol (Bytes)","TCP DL Retrans. Vol (Bytes)" ,"count", "frequency of TCP DL", 10, order=False )

print("Top 10 TCP DL in the dataset")
print(top_RTT_DL,"\n")
print("Bottom 10 TCP DL in the dataset")
print(bottom_RTT_DL,"\n")
print("Top 10 frequent TCP DL in the dataset")
print(frequent_RTT_DL)

Top 10 TCP UL in the dataset
   index  Top RTT UL
0    303    7,120.00
1    663    5,607.00
2    543    5,422.00
3     41    5,279.00
4    567    4,677.00
5    404    4,226.00
6    437    4,021.00
7    674    3,923.00
8    627    3,093.00
9    423    3,029.00 

Bottom 10 TCP UL in the dataset
   index  Bottom RTT UL
0     14           0.00
1      8           1.00
2      5           2.00
3     18           3.00
4      3           4.00
5      0           5.00
6      7           6.00
7      2           7.00
8     40           8.00
9     44           9.00 

Top 10 frequent TCP UL in the dataset
   Avg RTT UL (ms)  frequency of TP UL
5             5.00               42475
2             2.00               14277
1             1.00               12826
4             4.00               10242
3             3.00                7935
6             6.00                7518
7             7.00                4196
0             0.00                4126
8             8.00                3209
9           

The distribution of the average throughput  per handset type and provide interpretation for your findings.
The average TCP retransmission view per handset type and provide interpretation for your findings.


### The distribution of the average throughput per handset type

In [58]:
print("Avg Tp DL per handset type","\n")
TP_DL_handset = utilities.find_agg(cleaned_data, "Handset Type","Avg Bearer TP DL (kbps)" ,"sum", "Avg TP DL for handset type", 10, order=False )
print(TP_DL_handset,"\n\n")

print("Avg Tp UL per handset type","\n")
TP_UL_handset = utilities.find_agg(cleaned_data, "Handset Type","Avg Bearer TP UL (kbps)" ,"sum", "Avg TP UL for handset type", 10, order=False )
print(TP_UL_handset)


Avg Tp DL per handset type 

                     Handset Type  Avg TP DL for handset type
316              Huawei B528S-23A              809,552,425.00
59         Apple iPhone 7 (A1778)               68,206,317.00
53        Apple iPhone 6S (A1688)               65,169,580.00
49         Apple iPhone 6 (A1586)               59,387,996.00
955  Samsung Galaxy S8 (Sm-G950F)               57,632,886.00
337                  Huawei E5180               55,039,474.00
78        Apple iPhone Xr (A2105)               52,989,435.00
66         Apple iPhone 8 (A1905)               50,359,567.00
75         Apple iPhone X (A1901)               49,328,216.00
69    Apple iPhone 8 Plus (A1897)               33,537,386.00 


Avg Tp UL per handset type 

                     Handset Type  Avg TP UL for handset type
316              Huawei B528S-23A              121,478,363.00
78        Apple iPhone Xr (A2105)                9,446,673.00
49         Apple iPhone 6 (A1586)                9,347,012.00
337      

In [64]:
# sn.catplot(data = cleaned_data,x=cleaned_data["Avg Bearer TP DL (kbps)"].count(),y="Handset Type",kind="bar")

### The average TCP retransmission view per handset type

In [67]:
print("Avg TCp DL per handset type","\n")
TCP_DL_handset = utilities.find_agg(cleaned_data, "Handset Type","TCP DL Retrans. Vol (Bytes)" ,"sum", "Avg TCP DL for handset type", 10, order=False )
print(TCP_DL_handset,"\n\n")

print("Avg Tp UL per handset type","\n")
TCP_UL_handset = utilities.find_agg(cleaned_data, "Handset Type","TCP UL Retrans. Vol (Bytes)" ,"sum", "Avg TCP UL for handset type", 10, order=False )
print(TCP_UL_handset)


Avg TCp DL per handset type 

                     Handset Type  Avg TCP DL for handset type
316              Huawei B528S-23A         1,045,218,364,148.00
337                  Huawei E5180            61,750,921,075.00
341                  Huawei E5573            15,790,482,356.00
53        Apple iPhone 6S (A1688)            15,571,283,379.00
49         Apple iPhone 6 (A1586)            14,440,825,042.00
78        Apple iPhone Xr (A2105)            10,785,100,111.00
59         Apple iPhone 7 (A1778)            10,734,496,076.00
75         Apple iPhone X (A1901)             9,001,491,315.00
955  Samsung Galaxy S8 (Sm-G950F)             8,261,001,366.00
66         Apple iPhone 8 (A1905)             7,662,764,050.00 


Avg Tp UL per handset type 

                     Handset Type  Avg TCP UL for handset type
316              Huawei B528S-23A            33,637,124,455.00
337                  Huawei E5180             1,848,022,017.50
75         Apple iPhone X (A1901)               625,061,

Using the experience metrics above, perform a k-means clustering (where k = 3)
we have 3 metrics and they are TCP retransmission UL/DL, throughput UL/DL and RTT UL/DL

In [86]:
cleaned_copy = cleaned_data.copy()
df_experience= cleaned_copy.groupby("MSISDN/Number")

df_RTT_DL = pd.DataFrame(df_experience["Avg RTT DL (ms)"].sum())
df_RTT_UL = pd.DataFrame(df_experience["Avg RTT UL (ms)"].sum())
df_TP_DL = pd.DataFrame(df_experience["Avg Bearer TP DL (kbps)"].sum())
df_TP_UL = pd.DataFrame(df_experience["Avg Bearer TP UL (kbps)"].sum())
df_TCP_DL = pd.DataFrame(df_experience["TCP DL Retrans. Vol (Bytes)"].sum())
df_TCP_UL = pd.DataFrame(df_experience["TCP UL Retrans. Vol (Bytes)"].sum())

df_user_experience= pd.DataFrame({"RTT DL": df_RTT_DL["Avg RTT DL (ms)"],"RTT UL": df_RTT_UL["Avg RTT UL (ms)"],"TCP DL": df_TCP_DL["TCP DL Retrans. Vol (Bytes)"],"TCP UL": df_TCP_UL["TCP UL Retrans. Vol (Bytes)"],"TP DL": df_TP_DL["Avg Bearer TP DL (kbps)"],"TP UL": df_TP_UL["Avg Bearer TP UL (kbps)"]})
df_user_experience


,RTT DL,RTT UL,TCP DL,TCP UL,TP DL,TP UL
MSISDN/Number,,,,,,
"33,601,001,722.00",46.00,0.00,"568,730.00","20,949.50",37.00,39.00
"33,601,001,754.00",30.00,1.00,"568,730.00","20,949.50",48.00,51.00
"33,601,002,511.00",45.00,5.00,"568,730.00","20,949.50",48.00,49.00
"33,601,007,832.00",69.00,15.00,"1,066.00","20,949.50",204.00,44.00
"33,601,008,617.00",114.00,5.00,"9,918,360.00","42,151.50","40,395.00","16,449.00"
...,...,...,...,...,...,...
"33,789,996,170.00",42.00,10.00,"215,044.00","3,001.00","9,978.00",387.00
"33,789,997,247.00",34.00,6.00,"568,730.00","20,949.50",68.00,48.00
"3,197,020,876,596.00",45.00,5.00,"568,730.00","20,949.50",1.00,0.00


In [78]:
normalized_RTT_DL = norm_scaling.normalize(df_RTT_DL)
normalized_RTT_UL = norm_scaling.normalize(df_RTT_UL)
normalized_TP_DL = norm_scaling.normalize(df_TP_DL)
normalized_TP_UL = norm_scaling.normalize(df_TP_UL)
normalized_TCP_DL = norm_scaling.normalize(df_TCP_DL)
normalized_TCP_UL = norm_scaling.normalize(df_TCP_UL)
X = np.hstack((normalized_RTT_DL,normalized_RTT_UL,normalized_TP_DL,normalized_TP_UL,normalized_TCP_DL,normalized_TCP_UL))
X = norm_scaling.normalize(X)

In [79]:
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
import pickle

In [80]:
kmeans = KMeans(n_clusters=3).fit(X)
pickle.dump(kmeans, open("../models/user_exper_model.pkl", 'wb'))
kmeans = kmeans.predict(X)
X.shape

(106856, 6)

In [88]:
# organizing datapoints per cluster
un_normalized_data = df_user_experience.to_numpy()
clusters = {}
cluster_no = 3
container = []
for i in range(0,cluster_no):
    for x,y in zip(un_normalized_data,kmeans):    
        if y == i:
            container.append(x)
    clusters["cluster_"+str(i)] = np.array(container)
    container = []

In [89]:
# printing information for each cluster
metrics = 4
metrics_name = ["RTT DL","RTT UL","TCP DL","TCP UL","TP DL","TP UL"]
for i in range(3):
    for metric in range(0,metrics):
        data_cluster = clusters["cluster_"+str(i)][:,metric]
        print("min of cluster "+str(i)+ ", "+metrics_name[metric],data_cluster.min())
        print("max of cluster "+str(i)+ ", "+metrics_name[metric],data_cluster.max())
        print("avg of cluster "+str(i)+ ", "+metrics_name[metric],data_cluster.mean())

min of cluster 0, RTT DL 0.0
max of cluster 0, RTT DL 96923.0
avg of cluster 0, RTT DL 134.20729076790337
min of cluster 0, RTT UL 0.0
max of cluster 0, RTT UL 7120.0
avg of cluster 0, RTT UL 21.139860074276925
min of cluster 0, TCP DL 2.0
max of cluster 0, TCP DL 1454975201.0
avg of cluster 0, TCP DL 6202603.855506997
min of cluster 0, TCP UL 1.0
max of cluster 0, TCP UL 5168607636.5
avg of cluster 0, TCP UL 386553.5843165022
min of cluster 1, RTT DL 31.0
max of cluster 1, RTT DL 2721.0
avg of cluster 1, RTT DL 265.93392070484583
min of cluster 1, RTT UL 7.0
max of cluster 1, RTT UL 1449.0
avg of cluster 1, RTT UL 103.9295154185022
min of cluster 1, TCP DL 1472698725.0
max of cluster 1, TCP DL 8332732777.0
avg of cluster 1, TCP DL 2931273374.8854628
min of cluster 1, TCP UL 2.0
max of cluster 1, TCP UL 807925207.0
avg of cluster 1, TCP UL 5338808.162995595
min of cluster 2, RTT DL 295644.0
max of cluster 2, RTT DL 295644.0
avg of cluster 2, RTT DL 295644.0
min of cluster 2, RTT UL 195